<a href="https://colab.research.google.com/github/AlvingEj/Arvenses/blob/main/Clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import zipfile
import os

# Subir el archivo zip
uploaded = files.upload()

# Asegurarse de que el archivo zip esté presente en la subida
if uploaded:
    # Obtener el nombre del archivo zip subido
    zip_filename = list(uploaded.keys())[0]

    # Descomprimir el archivo zip
    try:
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall('dataset')
        print("Archivo zip descomprimido correctamente")
    except zipfile.BadZipFile:
        print("Error: El archivo subido no es un archivo zip válido.")
else:
    print("Error: No se subió ningún archivo zip.")


In [ ]:
# Verificar la estructura de directorios
print("Contenido de la carpeta 'dataset':")
print(os.listdir('dataset'))

print("Contenido de la carpeta 'dataset/dataset':")
print(os.listdir('dataset/dataset'))

print("Contenido de la carpeta 'dataset/dataset/Manzana':")
print(os.listdir('dataset/dataset/Manzana'))
print("Contenido de la carpeta 'dataset/dataset/Uva':")
print(os.listdir('dataset/dataset/Uva'))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Generadores de datos
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Generadores de datos para la clasificación del tipo de planta
train_generator_type = datagen.flow_from_directory(
    'dataset/dataset',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator_type = datagen.flow_from_directory(
    'dataset/dataset',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Generadores de datos para estado de las plantas para Manzana
train_generator_status_A = datagen.flow_from_directory(
    'dataset/dataset/Manzana',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator_status_A = datagen.flow_from_directory(
    'dataset/dataset/Manzana',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Generadores de datos para estado de las plantas para Uva
train_generator_status_B = datagen.flow_from_directory(
    'dataset/dataset/Uva',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator_status_B = datagen.flow_from_directory(
    'dataset/dataset/Uva',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)



In [ ]:
# Modelo de Clasificación de Tipo de Planta
model_type = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilar el modelo
model_type.compile(loss='binary_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy'])

# Resumen del modelo
model_type.summary()

# Entrenar el modelo
history_type = model_type.fit(
    train_generator_type,
    epochs=25,
    validation_data=validation_generator_type
)


In [ ]:
# Modelo de Clasificación de Estado de la Planta para Manzana
model_status_A = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model_status_A.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

model_status_A.summary()

history_status_A = model_status_A.fit(
    train_generator_status_A,
    epochs=25,
    validation_data=validation_generator_status_A
)


In [ ]:
# Modelo de Clasificación de Estado de la Planta para Uva
model_status_B = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model_status_B.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

model_status_B.summary()

history_status_B = model_status_B.fit(
    train_generator_status_B,
    epochs=25,
    validation_data=validation_generator_status_B
)


In [ ]:
from google.colab import files
import tensorflow as tf
import numpy as np

# Define los mapeos de índices a etiquetas para uva y manzana
estado_mapeo_uv = {
    0: 'Uva Podredumbre Negra',
    1: 'Uva Sana',
    2: 'Uva Sarampión Negro',
    3: 'Uva Tizón Hoja'
}

estado_mapeo_ma = {
    0: 'Manzana Podredumbre Negra',
    1: 'Manzana Roya Manzano',
    2: 'Manzana Sana',
    3: 'Manzana Sarna Manzano'
}

# Subir archivos
uploaded = files.upload()

# Iterar sobre todas las imágenes cargadas
for filename in uploaded.keys():
    img_path = filename

    # Preprocesar la imagen
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, axis=0) / 255.0

    # Predecir el tipo de planta
    plant_type = model_type.predict(img_array)
    plant_type_label = 1 if plant_type[0][0] >= 0.5 else 0

    # Predecir el estado de la planta basado en el tipo
    if plant_type_label == 0:
        status = model_status_A.predict(img_array)
        estado_mapeo = estado_mapeo_ma
    else:
        status = model_status_B.predict(img_array)
        estado_mapeo = estado_mapeo_uv

    # Obtener la etiqueta del estado de la planta
    status_label_index = tf.argmax(status, axis=1).numpy()[0]
    status_label = estado_mapeo.get(status_label_index, 'Desconocido')

    # Imprimir resultados
    plant_type_name = "Manzana" if plant_type_label == 0 else "Uva"
    print(f"{filename}: Tipo de planta - {plant_type_name}, Estado de la planta - {status_label}")
